In [4]:
import requests
from kafka import KafkaProducer
import json

def send_plane_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from the api    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])
        
        # Publish each entry to Kafka
       
        for state in states:
            output = {}
            for key in fields.keys():
                output[key] = state[fields[key]]
            # output["icao24"] = state[0]  # Id of the plane  (ICAO24)
            # output["origin_country"] = state[2]
            # output["velocity"] = state[9]
            # output["timestamp"] = state[3]  # Timestamp de la dernière position
            # output["latitude"] = state[6]  # Latitude de l'avion
            # output["longitude"] = state[5]  # Longitude de l'avion
            producer.send(topic, value=output)
        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(states)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [ ]:
from datetime import datetime, timedelta
fields = {
    "icao24":0,
    "origin_country":1,
    "velocity":9,
    "timestamp":3,
    "latitude":6,
    "longitude":5
}

api_url = "https://opensky-network.org/api/states/all?lamin=40.0&lamax=52.0&lomin=5&lomax=10.0"

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)

start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

In [8]:
import time 
for i in range(10):
    print(f"{start}-{end}")        
    send_plane_to_kafka("planes", api_url, fields)
    time.sleep(5)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = initial_date.strftime(date_format)
    end = end_date.strftime(date_format)

2025-01-25 17:00:00-2025-01-25 18:00:00
Sent 5640 records.
2025-01-25 18:00:00-2025-01-25 19:00:00


SSLError: HTTPSConnectionPool(host='opensky-network.org', port=443): Max retries exceeded with url: /api/states/all (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1010)')))

In [9]:
import requests
response = requests.get("https://opensky-network.org/api/states/all?lamin=40.0&lamax=52.0&lomin=5&lomax=10.0")

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en dictionnaire
    
    # Vérifier si "states" existe dans les données
    if "states" in data and data["states"]:
        for state in data["states"]:
            icao24 = state[0]  # Identifiant unique de l'avion (ICAO24)
            origin_country = state[2]
            velocity = state[9]
            timestamp = state[3]  # Timestamp de la dernière position
            latitude = state[6]  # Latitude de l'avion
            longitude = state[5]  # Longitude de l'avion

            # Afficher uniquement si latitude et longitude sont disponibles
            if latitude is not None and longitude is not None:
                print(f"ID: {icao24}, Timestamp: {timestamp}, Latitude: {latitude}, Longitude: {longitude}, Velocity : {velocity}, Origin : {origin_country}")
    else:
        print("Aucune donnée d'avion disponible.")
else:
    print("Erreur :", response.text)

ID: 4b1816, Timestamp: 1742291096, Latitude: 47.662, Longitude: 6.3705, Velocity : 238, Origin : Switzerland
ID: 4b1818, Timestamp: 1742291096, Latitude: 47.2994, Longitude: 8.1941, Velocity : 141.73, Origin : Switzerland
ID: 39de4c, Timestamp: 1742291096, Latitude: 42.4011, Longitude: 8.1192, Velocity : 225.91, Origin : France
ID: 440da6, Timestamp: 1742291096, Latitude: 44.6368, Longitude: 8.673, Velocity : 209.94, Origin : Austria
ID: 4b1806, Timestamp: 1742291069, Latitude: 46.2257, Longitude: 6.0939, Velocity : 0, Origin : Switzerland
ID: 4b1809, Timestamp: 1742291096, Latitude: 50.1432, Longitude: 9.194, Velocity : 230.75, Origin : Switzerland
ID: 4b1800, Timestamp: 1742291096, Latitude: 49.5706, Longitude: 6.3757, Velocity : 134.72, Origin : Switzerland
ID: 4952cd, Timestamp: 1742291096, Latitude: 45.6779, Longitude: 5.5197, Velocity : 227.31, Origin : Portugal
ID: 39de61, Timestamp: 1742291002, Latitude: 46.438, Longitude: 5.7938, Velocity : 40.52, Origin : France
ID: 399549, T